In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from pathlib import Path
import pickle
import scipy
import sys
sys.path.append('./src')
import algorithms_approx3 as LcGD

In [2]:
def retrieve_gen_model(dataset):
    pol = [x for x in dataset.split("__") if x.endswith("initial_pol")][0].split('_')[0]
    if len([x for x in dataset.split("__") if x.startswith("gaussian")]) > 0:
        return "gaussian", pol
    elif len([x for x in dataset.split("__") if x.startswith("uniform")]) > 0:
        return "uniform", pol
    else:
        return dataset.split("_")[0], pol

# Table Directed

In [350]:
# Folder Names
base = "./datasets/experiments/directed/approximation_large_task__degree_constraint"
#base = "./datasets/experiments/directed/approximation_curves_task__degree_constraint/"
#base = "./datasets/experiments/undirected/approximation_small_task__degree_constraint/"
ε = 1e-6

with open(Path(base) / Path("folder_names.txt"), "r") as file: # hashed names
    folders = [line.split("| ")[1][:-1] for line in file]
with open(Path(base) / Path("folder_names.txt"), "r") as file: # real names
    folders_real_names = [line.split("| ")[0] for line in file]

print("\\begin{table*}[]")
print("\\begin{tabular}{|llllllllll|}")
print(f"dataset & |V| & |E| & density & opinions & polarization & method   & time(s) & 1-obj/UB & 1-obj/INIT\\\\ ")
print("\\midrule")
data = {'Method': [], 'Distribution': [], 'Network': [], 'Polarity': [], 'Time': [],
        r'Overall $\rho$': [], r"Counterfactual $\rho$": []}
for code, dataset in zip(folders, folders_real_names):
    basepath = f"{base}/{code}"
    with open(f"{basepath}/experiments_data_method.pkl", "rb") as file:
        exps = pickle.load(file)
    #G_true = nx.read_gexf(f"{basepath}/graph_sample00_python.gexf", node_type=int)
    s = np.load(Path(basepath) / Path("internal_opinions_sample00_python.npy"))
    A_eq = scipy.sparse.load_npz(Path(basepath) / Path("A_eq_sparse_sample00_python.npz"))
    z_eq = np.load(Path(basepath) / Path("eq_opinions_sample00_python.npy"))
    n = len(s)
    e = A_eq.count_nonzero()
    
    # Table results
    max_n = 500
    print("\\toprule")
    print("\\toprule")
    printed_methods = []
    for i, d in exps.items():
        A = d['A']
        if 'routine' not in d['method_config']['params']:
            name = d['method_config']["name"] 
        else:
            name = d['method_config']['params']['routine']
        objectives = d['objectives']
        if name == "simple_GD":
            pass
        else:
            # First objective
            I = scipy.sparse.identity(A_eq.shape[0])
            z, _ = scipy.sparse.linalg.bicg(2*I - A_eq, s, tol=ε)
            z = z.reshape((len(s), 1))
            initial_PD_index = LcGD.objective_f(s, A_eq)

            # Natural objective (using input graph)
            obj_nat = LcGD.objective_f(z, A_eq)

            # Objective
            z, _ = scipy.sparse.linalg.bicg(2*I - A, s, tol=ε)
            z = z.reshape((len(s), 1))
            obj = LcGD.objective_f(z, A)

            #if d['method_config']['name'] == 'GD_optimizer':
            #    assert np.absolute(objectives[-1] - obj) < 1e-3, (objectives[-1], obj)


            # model data
            opin, pol = retrieve_gen_model(dataset)
            # Layout
            if name == "opposite_view_heuristic":
                name = "oppo-view"
            elif name == "popularity_heuristic":
                name = "pop"
            elif name == "simple_GD":
                name = "LcGD"
            elif name == "NAG":
                name = "NAG-LcGD"
            elif name == "ADAM":
                name = "LcGD"
            elif name == "neutral_view_heuristic":
                name = "neutral-view"
            
            if not name in printed_methods:
                D_perc = e/(n*(n-1))*100
                print(f"{dataset.split('_')[0]}&{int(n/1000)}k&{int(e/1000)}k&"
                      f"{D_perc:.5f}\%&{opin}&{pol}&{name} &{d['runtime']:.1f} & "
                      f"{(1-obj/obj_nat)*100:.1f}\% & {(1-obj/initial_PD_index)*100:.1f}\%\\\\")
                data['Method'].append(name)
                data['Distribution'].append(opin)
                data['Polarity'].append(pol)
                data['Network'].append(dataset.split('_')[0])
                data[r'Overall $\rho$'].append((1-obj/initial_PD_index)*100)
                data[r'Counterfactual $\rho$'].append((1-obj/obj_nat)*100)
                data['Time'].append(d['runtime'])
                printed_methods.append(name)
                #print(name, printed_methods)
            
print("\\bottomrule")
print("\end{tabular}")
print("\end{table*}")

\begin{table*}[]
\begin{tabular}{|llllllllll|}
dataset & |V| & |E| & density & opinions & polarization & method   & time(s) & 1-obj/UB & 1-obj/INIT\\ 
\midrule
\toprule
\toprule
brexit&7k&530k&1.00104\%&brexit&1&oppo-view &0.0 & 20.5\% & 80.2\%\\
brexit&7k&530k&1.00104\%&brexit&1&pop &0.0 & -1.6\% & 74.7\%\\
brexit&7k&530k&1.00104\%&brexit&1&neutral-view &0.0 & 19.4\% & 80.0\%\\
brexit&7k&530k&1.00104\%&brexit&1&LcGD &43.1 & 28.2\% & 82.2\%\\
\toprule
\toprule
brexit&7k&530k&1.00104\%&brexit&5.0&oppo-view &0.1 & 20.5\% & 80.2\%\\
brexit&7k&530k&1.00104\%&brexit&5.0&pop &0.0 & -1.6\% & 74.7\%\\
brexit&7k&530k&1.00104\%&brexit&5.0&neutral-view &0.0 & 19.4\% & 80.0\%\\
brexit&7k&530k&1.00104\%&brexit&5.0&LcGD &38.1 & 28.2\% & 82.2\%\\
\toprule
\toprule
vaxNoVax&10k&1430k&1.22021\%&vaxNoVax&1&oppo-view &0.2 & -3.7\% & 75.2\%\\
vaxNoVax&10k&1430k&1.22021\%&vaxNoVax&1&pop &0.1 & -0.4\% & 76.0\%\\
vaxNoVax&10k&1430k&1.22021\%&vaxNoVax&1&neutral-view &0.1 & 23.2\% & 81.6\%\\
vaxNoVax&10k&1430k

sbm&9k&86k&0.08634\%&gaussian&5.0&LcGD &128.0 & 46.5\% & 75.8\%\\
\toprule
\toprule
munmun&464k&834k&0.00039\%&uniform&1&oppo-view &0.0 & 3.4\% & 82.8\%\\
munmun&464k&834k&0.00039\%&uniform&1&pop &0.0 & -0.1\% & 82.2\%\\
munmun&464k&834k&0.00039\%&uniform&1&LcGD &49.5 & 6.9\% & 83.4\%\\
munmun&464k&834k&0.00039\%&uniform&1&neutral-view &0.0 & 1.0\% & 82.4\%\\
\toprule
\toprule
munmun&464k&834k&0.00039\%&gaussian&1&oppo-view &0.0 & 1.5\% & 80.6\%\\
munmun&464k&834k&0.00039\%&gaussian&1&pop &0.0 & -0.1\% & 80.3\%\\
munmun&464k&834k&0.00039\%&gaussian&1&neutral-view &0.0 & 3.8\% & 81.0\%\\
munmun&464k&834k&0.00039\%&gaussian&1&LcGD &384.0 & 9.8\% & 82.2\%\\
\toprule
\toprule
munmun&464k&834k&0.00039\%&uniform&5.0&oppo-view &0.0 & 6.0\% & 83.3\%\\
munmun&464k&834k&0.00039\%&uniform&5.0&pop &0.0 & -0.1\% & 82.2\%\\
munmun&464k&834k&0.00039\%&uniform&5.0&neutral-view &0.0 & -0.9\% & 82.0\%\\
munmun&464k&834k&0.00039\%&uniform&5.0&LcGD &368.9 & 7.5\% & 83.5\%\\
\toprule
\toprule
munmun&464k&8

epinions&73k&767k&0.01413\%&uniform&1&LcGD &243.4 & 15.9\% & 83.7\%\\
\toprule
\toprule
epinions&73k&767k&0.01413\%&gaussian&1&oppo-view &0.0 & 3.2\% & 78.6\%\\
epinions&73k&767k&0.01413\%&gaussian&1&pop &0.0 & 1.5\% & 78.2\%\\
epinions&73k&767k&0.01413\%&gaussian&1&neutral-view &0.0 & 16.6\% & 81.5\%\\
epinions&73k&767k&0.01413\%&gaussian&1&LcGD &740.1 & 22.9\% & 82.9\%\\
\toprule
\toprule
epinions&73k&767k&0.01413\%&uniform&5.0&oppo-view &0.0 & 15.7\% & 83.6\%\\
epinions&73k&767k&0.01413\%&uniform&5.0&pop &0.0 & -0.2\% & 80.5\%\\
epinions&73k&767k&0.01413\%&uniform&5.0&neutral-view &0.0 & -1.2\% & 80.3\%\\
epinions&73k&767k&0.01413\%&uniform&5.0&LcGD &220.3 & 16.9\% & 83.8\%\\
\toprule
\toprule
epinions&73k&767k&0.01413\%&gaussian&5.0&oppo-view &0.0 & 14.5\% & 72.9\%\\
epinions&73k&767k&0.01413\%&gaussian&5.0&pop &0.0 & -4.9\% & 66.8\%\\
epinions&73k&767k&0.01413\%&gaussian&5.0&neutral-view &0.0 & 22.0\% & 75.3\%\\
epinions&73k&767k&0.01413\%&gaussian&5.0&LcGD &282.0 & 38.2\% & 80.4\

In [357]:
#synthetic table
df = pd.DataFrame(data)
df = df[df.Distribution.isin(["gaussian", "brexit", "vaxNoVax"])]
df = df.drop(columns=['Distribution'])
latex_df=df.pivot_table(df, index=['Network', 'Polarity'],
            columns="Method").style.format(decimal='.', thousands="'", precision=2).to_latex(hrules=True)
with open("./draft/table_directed_gaussian.tex", "w") as file:
    file.write(latex_df)

In [358]:
#synthetic table
df = pd.DataFrame(data)
df = df[df.Distribution.isin(["uniform", "brexit", "vaxNoVax"])]
df = df.drop(columns=['Distribution'])
latex_df=df.pivot_table(df, index=['Network', 'Polarity'],
            columns="Method").style.format(decimal='.', thousands="'", precision=2).to_latex(hrules=True)
with open("./draft/table_directed_uniform.tex", "w") as file:
    file.write(latex_df)

# Table ablation

In [8]:
# Folder Names
base = "./datasets/experiments/directed/approximation_large_task__degree_constraint"
#base = "./datasets/experiments/directed/approximation_curves_task__degree_constraint/"
#base = "./datasets/experiments/undirected/approximation_small_task__degree_constraint/"
ε = 1e-6

with open(Path(base) / Path("folder_names.txt"), "r") as file: # hashed names
    folders = [line.split("| ")[1][:-1] for line in file]
with open(Path(base) / Path("folder_names.txt"), "r") as file: # real names
    folders_real_names = [line.split("| ")[0] for line in file]

print("\\begin{table*}[]")
print("\\begin{tabular}{|llllllllll|}")
print(f"dataset & |V| & |E| & density & opinions & polarization & method   & time(s) & 1-obj/UB & 1-obj/INIT\\\\ ")
print("\\midrule")
data = {'Method': [], 'Distribution': [], 'Network': [], 'Polarity': [], 'Time': [],
        r'Overall $\rho$': [], r"Counterfactual $\rho$": []}
for code, dataset in zip(folders, folders_real_names):
    basepath = f"{base}/{code}"
    with open(f"{basepath}/experiments_data_method.pkl", "rb") as file:
        exps = pickle.load(file)
    #G_true = nx.read_gexf(f"{basepath}/graph_sample00_python.gexf", node_type=int)
    s = np.load(Path(basepath) / Path("internal_opinions_sample00_python.npy"))
    A_eq = scipy.sparse.load_npz(Path(basepath) / Path("A_eq_sparse_sample00_python.npz"))
    z_eq = np.load(Path(basepath) / Path("eq_opinions_sample00_python.npy"))
    n = len(s)
    e = A_eq.count_nonzero()
    
    # Table results
    max_n = 500
    print("\\toprule")
    print("\\toprule")
    printed_methods = []
    for i, d in exps.items():
        A = d['A']
        if 'routine' not in d['method_config']['params']:
            name = d['method_config']["name"] 
        else:
            name = d['method_config']['params']['routine']
        objectives = d['objectives']
        if name in ["opposite_view_heuristic", "popularity_heuristic", "neutral_view_heuristic"]:
            pass
        else:
            # First objective
            I = scipy.sparse.identity(A_eq.shape[0])
            z, _ = scipy.sparse.linalg.bicg(2*I - A_eq, s, tol=ε)
            z = z.reshape((len(s), 1))
            initial_PD_index = LcGD.objective_f(s, A_eq)

            # Natural objective (using input graph)
            obj_nat = LcGD.objective_f(z, A_eq)

            # Objective
            z, _ = scipy.sparse.linalg.bicg(2*I - A, s, tol=ε)
            z = z.reshape((len(s), 1))
            obj = LcGD.objective_f(z, A)

            #if d['method_config']['name'] == 'GD_optimizer':
            #    assert np.absolute(objectives[-1] - obj) < 1e-3, (objectives[-1], obj)


            # model data
            opin, pol = retrieve_gen_model(dataset)
            # Layout
            if name == "simple_GD":
                name = "simple_GD"
            elif name == "NAG":
                name = "NAG-LcGD"
            elif name == "ADAM":
                name = "LcGD"
            
            if not name in printed_methods:
                D_perc = e/(n*(n-1))*100
                print(f"{dataset.split('_')[0]}&{int(n/1000)}k&{int(e/1000)}k&"
                      f"{D_perc:.5f}\%&{opin}&{pol}&{name} &{d['runtime']:.1f} & "
                      f"{(1-obj/obj_nat)*100:.1f}\% & {(1-obj/initial_PD_index)*100:.1f}\%\\\\")
                data['Method'].append(name)
                data['Distribution'].append(opin)
                data['Polarity'].append(pol)
                data['Network'].append(dataset.split('_')[0])
                data[r'Overall $\rho$'].append((1-obj/initial_PD_index)*100)
                data[r'Counterfactual $\rho$'].append((1-obj/obj_nat)*100)
                data['Time'].append(d['runtime'])
                #printed_methods.append(name)
                #print(name, printed_methods)
            
print("\\bottomrule")
print("\end{tabular}")
print("\end{table*}")

\begin{table*}[]
\begin{tabular}{|llllllllll|}
dataset & |V| & |E| & density & opinions & polarization & method   & time(s) & 1-obj/UB & 1-obj/INIT\\ 
\midrule
\toprule
\toprule
brexit&7k&530k&1.00104\%&brexit&1&simple_GD &88.8 & 28.2\% & 82.2\%\\
brexit&7k&530k&1.00104\%&brexit&1&LcGD &43.1 & 28.2\% & 82.2\%\\
\toprule
\toprule
brexit&7k&530k&1.00104\%&brexit&5.0&simple_GD &79.2 & 28.2\% & 82.2\%\\
brexit&7k&530k&1.00104\%&brexit&5.0&LcGD &38.1 & 28.2\% & 82.2\%\\
\toprule
\toprule
vaxNoVax&10k&1430k&1.22021\%&vaxNoVax&1&simple_GD &198.4 & 29.0\% & 83.0\%\\
vaxNoVax&10k&1430k&1.22021\%&vaxNoVax&1&LcGD &95.0 & 29.8\% & 83.2\%\\
\toprule
\toprule
vaxNoVax&10k&1430k&1.22021\%&vaxNoVax&5.0&simple_GD &193.5 & 29.0\% & 83.0\%\\
vaxNoVax&10k&1430k&1.22021\%&vaxNoVax&5.0&LcGD &101.0 & 29.8\% & 83.2\%\\
\toprule
\toprule
barabasi&7k&10k&0.02168\%&uniform&1&simple_GD &10.6 & 6.3\% & 79.4\%\\
barabasi&7k&10k&0.02168\%&uniform&1&LcGD &15.6 & 10.5\% & 80.3\%\\
\toprule
\toprule
barabasi&4k&8k&0.03

\toprule
\toprule
digg-friends&231k&1666k&0.00312\%&gaussian&1&simple_GD &344.9 & 8.7\% & 80.9\%\\
digg-friends&231k&1666k&0.00312\%&gaussian&1&LcGD &639.5 & 18.1\% & 82.9\%\\
\toprule
\toprule
digg-friends&231k&1666k&0.00312\%&uniform&5.0&simple_GD &330.4 & 7.1\% & 82.5\%\\
digg-friends&231k&1666k&0.00312\%&uniform&5.0&LcGD &369.0 & 13.6\% & 83.7\%\\
\toprule
\toprule
digg-friends&231k&1666k&0.00312\%&gaussian&5.0&simple_GD &505.1 & 32.4\% & 78.7\%\\
digg-friends&231k&1666k&0.00312\%&gaussian&5.0&LcGD &354.5 & 31.0\% & 78.3\%\\
\toprule
\toprule
epinions&73k&767k&0.01413\%&uniform&1&simple_GD &875.8 & 16.1\% & 83.7\%\\
epinions&73k&767k&0.01413\%&uniform&1&LcGD &243.4 & 15.9\% & 83.7\%\\
\toprule
\toprule
epinions&73k&767k&0.01413\%&gaussian&1&simple_GD &441.4 & 20.2\% & 82.3\%\\
epinions&73k&767k&0.01413\%&gaussian&1&LcGD &740.1 & 22.9\% & 82.9\%\\
\toprule
\toprule
epinions&73k&767k&0.01413\%&uniform&5.0&simple_GD &504.0 & 16.9\% & 83.8\%\\
epinions&73k&767k&0.01413\%&uniform&5.0&Lc

In [21]:
#synthetic table
import pandas as pd
df = pd.DataFrame(data)
df = df[df.Distribution.isin(["gaussian", "brexit", "vaxNoVax"])]
df = df.drop(columns=['Distribution'])
df = df[df.Polarity=='5.0']
df = df.drop(columns=['Polarity'])
df = df.drop(columns=[r"Counterfactual $\rho$"])
latex_df=df.pivot_table(df, index=['Network'],
            columns="Method").style.format(decimal='.', thousands="'", precision=2).to_latex(hrules=True)
with open("./draft/table_directed_ablation_gaussian.tex", "w") as file:
    file.write(latex_df)

# Table Undirected

In [383]:
# Folder Names
#base = "./datasets/experiments/directed/approximation_large_task__degree_constraint"
#base = "./datasets/experiments/directed/approximation_curves_task__degree_constraint/"
base = "./datasets/experiments/undirected/approximation_small_task__degree_constraint/"
ε = 1e-6

with open(Path(base) / Path("folder_names.txt"), "r") as file: # hashed names
    folders = [line.split("| ")[1][:-1] for line in file]
with open(Path(base) / Path("folder_names.txt"), "r") as file: # real names
    folders_real_names = [line.split("| ")[0] for line in file]

print("\\begin{table*}[]")
print("\\begin{tabular}{|llllllllll|}")
print(f"dataset & |V| & |E| & density & opinions & polarization & method   & time(s) & 1-obj/UB & 1-obj/INIT\\\\ ")
print("\\midrule")
data2 = {'Method': [], 'Distribution': [], 'Network': [], 'Polarity': [], 'Time': [],
        r'Overall $\rho$': [], r"Counterfactual $\rho$": [], 'nodes': []}
for code, dataset in zip(folders, folders_real_names):
    basepath = f"{base}/{code}"
    with open(f"{basepath}/experiments_data_method.pkl", "rb") as file:
        exps = pickle.load(file)
    #G_true = nx.read_gexf(f"{basepath}/graph_sample00_python.gexf", node_type=int)
    s = np.load(Path(basepath) / Path("internal_opinions_sample00_python.npy"))
    A_eq = scipy.sparse.load_npz(Path(basepath) / Path("A_eq_sparse_sample00_python.npz"))
    z_eq = np.load(Path(basepath) / Path("eq_opinions_sample00_python.npy"))
    n = len(s)
    e = A_eq.count_nonzero()
    
    # Table results
    max_n = 500
    print("\\toprule")
    print("\\toprule")
    printed_methods = []
    for i, d in exps.items():
        A = d['A']
        if 'routine' not in d['method_config']['params']:
            name = d['method_config']["name"] 
        else:
            name = d['method_config']['params']['routine']
        objectives = d['objectives']
        if name == "simple_GD":
            pass
        else:
            # First objective
            I = scipy.sparse.identity(A_eq.shape[0])
            z, _ = scipy.sparse.linalg.bicg(2*I - A_eq, s, tol=ε)
            z = z.reshape((len(s), 1))
            initial_PD_index = LcGD.objective_f(s, A_eq)

            # Natural objective (using input graph)
            obj_nat = LcGD.objective_f(z, A_eq)

            # Objective
            z, _ = scipy.sparse.linalg.bicg(2*I - A, s, tol=ε)
            z = z.reshape((len(s), 1))
            obj = LcGD.objective_f(z, A)

            #if d['method_config']['name'] == 'GD_optimizer':
            #    assert np.absolute(objectives[-1] - obj) < 1e-3, (objectives[-1], obj)


            # model data
            opin, pol = retrieve_gen_model(dataset)
            # Layout
            if name == "opposite_view_heuristic":
                name = "oppo-view"
            elif name == "popularity_heuristic":
                name = "pop"
            elif name == "simple_GD":
                name = "LcGD"
            elif name == "NAG":
                name = "NAG-LcGD"
            elif name == "ADAM":
                name = "LcGD"
            elif name == "neutral_view_heuristic":
                name = "neutral-view"
            
            if not name in printed_methods:
                D_perc = e/(n*(n-1))*100
                print(f"{dataset.split('_')[0]}&{int(n/1000)}k&{int(e/1000)}k&"
                      f"{D_perc:.5f}\%&{opin}&{pol}&{name} &{d['runtime']:.1f} & "
                      f"{(1-obj/obj_nat)*100:.1f}\% & {(1-obj/initial_PD_index)*100:.1f}\%\\\\")
                data2['Method'].append(name)
                data2['Distribution'].append(opin)
                data2['Polarity'].append(pol)
                data2['Network'].append(dataset.split('_')[0])
                data2['nodes'].append(n)
                data2[r'Overall $\rho$'].append((1-obj/initial_PD_index)*100)
                data2[r'Counterfactual $\rho$'].append((1-obj/obj_nat)*100)
                data2['Time'].append(d['runtime'])
                printed_methods.append(name)
                #print(name, printed_methods)
            
print("\\bottomrule")
print("\end{tabular}")
print("\end{table*}")

\begin{table*}[]
\begin{tabular}{|llllllllll|}
dataset & |V| & |E| & density & opinions & polarization & method   & time(s) & 1-obj/UB & 1-obj/INIT\\ 
\midrule
\toprule
\toprule
erdos&0k&0k&4.54545\%&uniform&5.0&LcGD &1.0 & 19.8\% & 85.8\%\\
erdos&0k&0k&4.54545\%&uniform&5.0&SCS &16.9 & 18.3\% & 85.6\%\\
\toprule
\toprule
erdos&0k&0k&5.31313\%&gaussian&5.0&LcGD &1.7 & 41.0\% & 76.8\%\\
erdos&0k&0k&5.31313\%&gaussian&5.0&SCS &57.4 & 29.0\% & 72.1\%\\
\toprule
\toprule
barabasi&0k&0k&7.75758\%&uniform&5.0&LcGD &1.8 & 20.3\% & 85.7\%\\
barabasi&0k&0k&7.75758\%&uniform&5.0&SCS &47.9 & 19.9\% & 85.6\%\\
\toprule
\toprule
barabasi&0k&0k&7.75758\%&gaussian&5.0&LcGD &3.7 & 36.6\% & 81.5\%\\
barabasi&0k&0k&7.75758\%&gaussian&5.0&SCS &73.0 & 30.9\% & 79.9\%\\
\toprule
\toprule
cs&0k&0k&19.28962\%&uniform&5.0&LcGD &2.5 & 27.7\% & 82.5\%\\
cs&0k&0k&19.28962\%&uniform&5.0&SCS &12.3 & 28.7\% & 82.7\%\\
\toprule
\toprule
cs&0k&0k&19.28962\%&gaussian&5.0&LcGD &3.9 & 40.1\% & 76.5\%\\
cs&0k&0k&19.28962

In [394]:
#synthetic table
df = pd.DataFrame(data2)
df = df.drop(columns=['Polarity', 'Time', r'Counterfactual $\rho$'])
latex_df=df.pivot_table(df, index=['Network', 'nodes'],
  columns=["Distribution", "Method"]).style.format(decimal='.', thousands="'", precision=2).to_latex(hrules=True)
with open("./draft/table_undirected.tex", "w") as file:
    file.write(latex_df)

In [393]:
df.pivot_table(df, index=['Network'],
                columns=["Distribution","Method"]).style.format(decimal='.', thousands="'", precision=2)

# Table Graph Stats

In [373]:
base = "./datasets/experiments/directed/approximation_large_task__degree_constraint"
#base = "./datasets/experiments/undirected/approximation_small_task__degree_constraint/"
ε = 1e-6

with open(Path(base) / Path("folder_names.txt"), "r") as file: # hashed names
    folders = [line.split("| ")[1][:-1] for line in file]
with open(Path(base) / Path("folder_names.txt"), "r") as file: # real names
    folders_real_names = [line.split("| ")[0] for line in file]


data = {'Network': [], 'nodes': [], 'edges': [], r'density \%': []}
for code, dataset in zip(folders, folders_real_names):
    basepath = f"{base}/{code}"
    name = dataset.split('_')[0]
    if name in data['Network']:
        continue
    print(dataset)
    s = np.load(Path(basepath) / Path("internal_opinions_sample00_python.npy"))
    A_eq = scipy.sparse.load_npz(Path(basepath) / Path("A_eq_sparse_sample00_python.npz"))
    z_eq = np.load(Path(basepath) / Path("eq_opinions_sample00_python.npy"))
    n = len(s)
    e = A_eq.count_nonzero()
    data['Network'].append(name)
    data['nodes'].append(n)
    data['edges'].append(e)
    data[r'density \%'].append(e/(n*(n-1))*100)

brexit_net_model__7281_nodes__None_None__100_percbudget__brexit_opinion_model__1_initial_pol__degree_constraint__0
vaxNoVax_net_model__10829_nodes__None_None__100_percbudget__vaxNoVax_opinion_model__1_initial_pol__degree_constraint__0
barabasi_net_model__7047_nodes__2_m__100_percbudget__uniform_opinion_model__1_initial_pol__degree_constraint__0
erdos_net_model__10000_nodes__0.0016_p__100_percbudget__uniform_opinion_model__1_initial_pol__degree_constraint__0
sbm_net_model__9999_nodes__0.1_β__100_percbudget__uniform_opinion_model__1_initial_pol__degree_constraint__0
munmun_twitter_social_net_model__464882_nodes__None_None__100_percbudget__uniform_opinion_model__1_initial_pol__degree_constraint__0
soc-epinions1_net_model__49604_nodes__None_None__100_percbudget__uniform_opinion_model__1_initial_pol__degree_constraint__0
facebook-wosn-wall_net_model__37082_nodes__None_None__100_percbudget__uniform_opinion_model__1_initial_pol__degree_constraint__0
ego-twitter_net_model__11548_nodes__None_No

In [379]:
df = pd.DataFrame(data)
df['description'] = ''
latex_df = df.style.format(decimal='.', thousands="'", precision=2).to_latex(hrules=True)
with open("./draft/table_stats.tex", "w") as file:
    file.write(latex_df)

In [369]:
data

{'Network': ['brexit'],
 'nodes': [7281],
 'edges': [530608],
 'density \\%': [1.0010398885553395]}

In [365]:
D_perc = e/(n*(n-1))*100

In [366]:
D_perc

1.0010398885553395

In [361]:
!ls ./datasets/experiments/directed/approximation_large_task__degree_constraint/e501954d5084ce63facee2f848a46b5b

A_eq_sparse_sample00_python.npz  generative_parameters.pkl
eq_opinions_sample00_python.npy  graph_sample00_python.gexf
experiments_data_method.pkl	 internal_opinions_sample00_python.npy


----

# removing results of baselines

In [310]:
!ls ./datasets/experiments/directed/approximation_large_task__degree_constraint/e501954d5084ce63facee2f848a46b5b

A_eq_sparse_sample00_python.npz  generative_parameters.pkl
eq_opinions_sample00_python.npy  graph_sample00_python.gexf
experiments_data_method.pkl	 internal_opinions_sample00_python.npy


In [318]:

# retrieveing deleted simulations
problematic_folders = []
for code, dataset in zip(folders, folders_real_names):
    basepath = f"{base}/{code}"
    with open(basepath+'/experiments_data_method.pkl', "rb") as file:
        method_name2data = pickle.load(file)
    existing_routines = []
    for _, x in method_name2data.items():
        if x['method_config']['name'] == "GD_optimizer":
            existing_routines.append(x['method_config']['params']['routine'])
    if 'ADAM' not in existing_routines:
        problematic_folders.append(code)

In [319]:
problematic_folders

['acd7c8692a3b1a35949053e75c71ba8e',
 'b54ea830a185aa7b7ff8ddacb3bbfe86',
 '64f1f2c6812c684e1bacffe78333bbec',
 'fe604e59883343446a170f9da09d1c99',
 'fb7ee542db25d2ca4bea86462e491550',
 '9f4804d7d7b5d9c96145186970cfecff',
 '5a3db83a8b7ed971f4a0e07216fb1bbc',
 '2f788949dbd645f4707b393678ecd5d8',
 '4a0076d84c9fd8e66c7afccc24e437b3',
 'b71dd8d9413d37d2fae9136f15302e4d',
 '351606b97ea8aaf8b38a8d410fa0993c',
 '896ddc67f4c88a7d30853e5f3768263f',
 '8d342dd274880aebb3478a66d3e44885',
 '4fa0e45f2aee2beeab77ae5ad7bf72b9',
 '2ebc4acbbc5a52f014a6c9e1dbd18c88',
 'ce54c3418f23d5cfb3e1b9cbe5b162ea',
 'cf63dca6cb51a8df646c703e2e8f8ba0',
 '951030e53870e40f788ab6eae670bae6',
 '7bf676cc5c5ae7f140e03ffe7b61671b',
 '9e1c934ba967c83bfe3ace30467dedd8',
 'b9739316f05093c1fcf5a619351a24f2',
 'f0e87cc432b740b48e934b46af326ecb',
 '1a73c76a5fa30e659701599b45ecbc73',
 'a97dca3e9f58956138b3f07c68972575',
 '8fcb32a4a57c803b4b72211e06e5fcf1',
 'a86f48172cb24c81c50a27cc98b405d7',
 'c3ccbab0a1a80bcf633608d22dfc60da',
 

In [293]:
basepath = "./datasets/experiments/directed/approximation_large_task__degree_constraint/"

folder = '8ba30ff78912162918bec3c0d4634738'
with open(basepath + f"{folder}/experiments_data_method.pkl", "rb") as file:
    experiments_data_method = pickle.load(file)
    for i, method_dict in experiments_data_method.items():
        print(i, method_dict['method_config']['name'])
        if method_dict['method_config']['name'] == "GD_optimizer":
            print(method_dict['method_config']['params'])
            #print(method_dict['method_config']['params']['grad_params']['budget'])

0 opposite_view_heuristic
1 popularity_heuristic
2 neutral_view_heuristic
3 GD_optimizer
{'routine': 'simple_GD', 'grad_params': {'budget': 1.0}, 'lr_params': {'lr_routine': 'constant', 'lr_coeff': 0.2}, 'early_stopping': 1, 'verbosity': 0}
4 GD_optimizer
{'routine': 'ADAM', 'grad_params': {'budget': 1.0}, 'lr_params': {'lr_routine': 'constant', 'lr_coeff': 0.2}, 'early_stopping': 1, 'verbosity': 0}


In [171]:
change_index  = 2
experiments_data_method.pop(change_index, None)

with open(basepath + f"{folder}/experiments_data_method.pkl", "wb") as file:
    pickle.dump(experiments_data_method, file)

# Misc

In [18]:
with open(f"./datasets/experiments/directed/modularity_task__degree_constraint/0098b54433afe5da7949f9c2857ad1a0/experiments_data_method.pkl", "rb") as file:
    _exps = pickle.load(file)

In [43]:
%%time
print("hello")

hello
CPU times: user 809 µs, sys: 9 µs, total: 818 µs
Wall time: 674 µs


In [36]:
_n = 25
print(f"{_exps[2]['runtime'] * 10 * _n * _n / 60 / 60} h")

5.763103564580281 h


In [41]:
np.linspace(.1, 1., num=25)

array([0.1   , 0.1375, 0.175 , 0.2125, 0.25  , 0.2875, 0.325 , 0.3625,
       0.4   , 0.4375, 0.475 , 0.5125, 0.55  , 0.5875, 0.625 , 0.6625,
       0.7   , 0.7375, 0.775 , 0.8125, 0.85  , 0.8875, 0.925 , 0.9625,
       1.    ])

In [42]:
np.linspace(.1, 10., num=25)

array([ 0.1   ,  0.5125,  0.925 ,  1.3375,  1.75  ,  2.1625,  2.575 ,
        2.9875,  3.4   ,  3.8125,  4.225 ,  4.6375,  5.05  ,  5.4625,
        5.875 ,  6.2875,  6.7   ,  7.1125,  7.525 ,  7.9375,  8.35  ,
        8.7625,  9.175 ,  9.5875, 10.    ])

In [ ]:
[.25, .5, 1., 2.5, 5.,]